<a href="https://colab.research.google.com/github/cibelerusso/MRASII/blob/main/C%C3%B3digos%20em%20Python/SME0823_Sele%C3%A7%C3%A3o_de_vari%C3%A1veis_Forward_e_Backward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SME0823 Modelos de Regressão e Aprendizado Supervisionado II

# Seleção de Variáveis - Métodos Forward e Backward


por **Cibele Russo**

**ICMC/USP - São Carlos SP**

## 1. Exemplo 1.14.1: Venda de Telhados (Seleção de Variáveis)

O Exemplo 1.14.1, intitulado **"Venda de Telhados"**, é uma aplicação da seleção de modelos em regressão linear múltipla.

### 1.1 Contexto e Variáveis

O estudo utiliza dados de $n=26$ filiais de uma rede de lojas de construção civil, referentes à venda de um tipo de telhado de madeira. A variável resposta é o **total de telhados vendidos**.

Quatro variáveis explicativas candidatas são consideradas para entrar no modelo:
*   **G** (*Gastos*): Gastos em propaganda.
*   **N** (*Nclientes*): Número de clientes cadastrados na loja.
*   **M** (*Marcas*): Número de marcas concorrentes do produto.
*   **P** (*Potencial*): Potencial da loja.

### Critérios de Seleção

Com quatro variáveis explicativas candidatas, o método de **Todas Regressões Possíveis** resulta em $T = 2^{(4)} = 16$ regressões possíveis a serem avaliadas.

A seleção do submodelo é baseada em quatro critérios principais, buscando o balanço entre o ajuste do modelo e a sua parcimônia:

1.  **Maior $R^2_k$** (Coeficiente de determinação).
2.  **Menor $s_k$** (Estimativa da variância residual).
3.  **$C_k \approx k$ e pequeno** (Estatística $C_k$ de Mallows, onde $k$ é o número de coeficientes).
4.  **Menor $Press_k$** (Estatística de Predição).



#### Análise de Diagnóstico e Escolha Final

Ler análise completa nas páginas 80 - 86 de Paula (2025)




In [15]:
import pandas as pd

url = "https://www.ime.usp.br/~giapaula/vendas.txt"

df = pd.read_csv(url, sep="\s+")

df


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1314876445.py:5: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(url, sep="\s+")


,telhados,gastos,clientes,marcas,potencial
0,79.3,5.5,31.0,10.0,8.0
1,200.1,2.5,55.0,8.0,6.0
2,163.2,8.0,67.0,12.0,9.0
3,200.1,3.0,50.0,7.0,16.0
4,146.0,3.0,38.0,8.0,15.0
5,177.7,2.9,71.0,12.0,17.0
6,30.9,8.0,30.0,12.0,8.0
7,291.9,9.0,56.0,5.0,10.0
8,160.0,4.0,42.0,8.0,4.0
9,339.4,6.5,73.0,5.0,16.0


In [16]:
df.columns

Index(['telhados', 'gastos', 'clientes', 'marcas', 'potencial'], dtype='object')

### Listar todas as combinações com k variáveis

In [17]:
from itertools import combinations
list(combinations(['gastos', 'clientes', 'marcas', 'potencial'], 1))

[('gastos',), ('clientes',), ('marcas',), ('potencial',)]

In [18]:
list(combinations(['gastos', 'clientes', 'marcas', 'potencial'], 2))

[('gastos', 'clientes'),
 ('gastos', 'marcas'),
 ('gastos', 'potencial'),
 ('clientes', 'marcas'),
 ('clientes', 'potencial'),
 ('marcas', 'potencial')]

In [19]:
list(combinations(['gastos', 'clientes', 'marcas', 'potencial'], 3))

[('gastos', 'clientes', 'marcas'),
 ('gastos', 'clientes', 'potencial'),
 ('gastos', 'marcas', 'potencial'),
 ('clientes', 'marcas', 'potencial')]

In [20]:
list(combinations(['gastos', 'clientes', 'marcas', 'potencial'], 4))

[('gastos', 'clientes', 'marcas', 'potencial')]

In [21]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Definição das variáveis para a Regressão
Y = df['telhados']
X = df[['gastos', 'clientes', 'marcas', 'potencial']]


In [22]:
# Função para o procedimento de 'Todas Regressões Possíveis'
# Avalia os modelos usando R2, s_k e AIC (análogo ao Ck de Mallows).
def all_subset_selection(X_data, Y_data, max_p=4):
    """Executa a seleção de subconjuntos e retorna as métricas."""
    features = list(X_data.columns)
    best_models = []

    for k in range(1, max_p + 1):
        for subset in combinations(features, k):
            X_subset = sm.add_constant(X_data[list(subset)], prepend=True)

            try:
                model = sm.OLS(Y_data, X_subset).fit()

                # Coletando métricas: k=número de parâmetros; sk=Erro Padrão Residual
                metrics = {
                    'Modelo': ' + '.join(['1'] + list(subset)),
                    'k': k + 1,
                    'R2': model.rsquared,
                    'sk': np.sqrt(model.mse_resid),
                    'AIC': model.aic,
                    'model_summary': model.summary()
                }
                best_models.append(metrics)
            except np.linalg.LinAlgError:
                continue

    results_df = pd.DataFrame(best_models)

    # Ordena pelo AIC (menor é melhor, análogo ao critério Ck ≈ k e pequeno)
    results_df = results_df.sort_values(by=['AIC', 'sk'], ascending=[True, True])

    return results_df

print("Resultados da Seleção de Todos os Subconjuntos")
selection_results = all_subset_selection(X, Y)
print(selection_results[['Modelo', 'k', 'R2', 'sk', 'AIC']])



Resultados da Seleção de Todos os Subconjuntos
                                        Modelo  k        R2         sk  \
10              1 + gastos + clientes + marcas  4  0.988912   9.490506   
7                        1 + clientes + marcas  3  0.987631   9.803376   
14  1 + gastos + clientes + marcas + potencial  5  0.989161   9.604406   
13           1 + clientes + marcas + potencial  4  0.987718   9.988382   
12             1 + gastos + marcas + potencial  4  0.775023  42.750101   
9                       1 + marcas + potencial  3  0.753388  43.774675   
2                                   1 + marcas  2  0.693857  47.745925   
5                          1 + gastos + marcas  3  0.710011  47.468617   
1                                 1 + clientes  2  0.612845  53.692842   
8                     1 + clientes + potencial  3  0.614970  54.696935   
4                        1 + gastos + clientes  3  0.613428  54.806329   
11           1 + gastos + clientes + potencial  4  0.616037  55.8

In [23]:
# Análise do Modelo Selecionado no Exemplo 1.14.1 (1 + gastos + clientes + marcas)

X_final_model = sm.add_constant(X[['gastos', 'clientes', 'marcas']], prepend=True)
model_final = sm.OLS(Y, X_final_model).fit()

print("\nEstimativas do Modelo 1 + gastos + clientes + marcas (Análise Inicial)")
print(model_final.summary())




Estimativas do Modelo 1 + gastos + clientes + marcas (Análise Inicial)
                            OLS Regression Results                            
Dep. Variable:               telhados   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     654.1
Date:                Wed, 05 Nov 2025   Prob (F-statistic):           1.20e-21
Time:                        22:41:28   Log-Likelihood:                -93.228
No. Observations:                  26   AIC:                             194.5
Df Residuals:                      22   BIC:                             199.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [24]:
# Forward Selection

# Usando PE = PS = 0.15, conforme sugerido em Paula (2025).
print("\n Procedimento Stepwise (Forward Selection) ")
import numpy as np
import statsmodels.api as sm

def stepwise_forward(X_data, Y_data, entry_p_value=0.15):
    remaining = list(X_data.columns)
    selected = [] # começa sem nenhuma variável

    print(f"\nProcedimento Stepwise (Forward Selection)")
    print(f"PE (Nível de Inclusão) = {entry_p_value}")

    while True:
        best_p_value = np.inf
        best_feature = None

        # Tenta adicionar a melhor variável restante
        for feature in remaining:
            current_subset = selected + [feature]
            X_current = sm.add_constant(X_data[current_subset], prepend=True)

            # Requisito: deve haver mais observações do que parâmetros
            n, p = X_current.shape
            if p >= n:
                continue

            model = sm.OLS(Y_data, X_current).fit()

            # P-valor da variável que acabou de ser adicionada
            if feature in model.pvalues.index:
                p_value = model.pvalues[feature]
            else:
                continue

            if p_value < best_p_value:
                best_p_value = p_value
                best_feature = feature

        # Critério de Parada/Inclusão (Pmin <= PE)
        if best_feature and best_p_value < entry_p_value:
            selected.append(best_feature)
            remaining.remove(best_feature)
            print(f"PASS: Adiciona '{best_feature}' (P-valor: {best_p_value:.4f}). "
                  f"Modelo atual: {' + '.join(['1'] + selected)}")
        else:
            break

    # Ajusta o modelo final
    if selected:
        print("\nModelo Final Selecionado pelo Stepwise")
        X_final = sm.add_constant(X_data[selected], prepend=True)
        final_model = sm.OLS(Y_data, X_final).fit()
        print(final_model.summary())
        return final_model
    else:
        print("Nenhuma variável foi selecionada.")
        return None


stepwise_forward(X, Y, entry_p_value=0.15)


 Procedimento Stepwise (Forward Selection) 

Procedimento Stepwise (Forward Selection)
PE (Nível de Inclusão) = 0.15
PASS: Adiciona 'marcas' (P-valor: 0.0000). Modelo atual: 1 + marcas
PASS: Adiciona 'clientes' (P-valor: 0.0000). Modelo atual: 1 + marcas + clientes
PASS: Adiciona 'gastos' (P-valor: 0.1252). Modelo atual: 1 + marcas + clientes + gastos

Modelo Final Selecionado pelo Stepwise
                            OLS Regression Results                            
Dep. Variable:               telhados   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     654.1
Date:                Wed, 05 Nov 2025   Prob (F-statistic):           1.20e-21
Time:                        22:41:28   Log-Likelihood:                -93.228
No. Observations:                  26   AIC:                             194.5
Df Residuals:                      22   BIC:        

In [25]:
## Backward elimination

import numpy as np
import statsmodels.api as sm

def stepwise_backward(X_data, Y_data, exit_p_value=0.15):
    """Procedimento Stepwise (Backward Elimination) - OLS"""
    selected = list(X_data.columns)  # começa com todas as variáveis
    print("\nProcedimento Stepwise (Backward Elimination)")
    print(f"PS (Nível de Exclusão) = {exit_p_value}")

    while True:
        # Ajusta o modelo com as variáveis atualmente selecionadas
        X_current = sm.add_constant(X_data[selected], prepend=True)
        model = sm.OLS(Y_data, X_current).fit()

        # P-valores das variáveis (exclui o intercepto)
        pvalues = model.pvalues.iloc[1:]
        worst_feature = pvalues.idxmax()
        worst_p_value = pvalues.max()

        # Critério de exclusão: remove se p > PS
        if worst_p_value > exit_p_value:
            selected.remove(worst_feature)
            print(f"DROP: Remove '{worst_feature}' (P-valor: {worst_p_value:.4f}). "
                  f"Modelo atual: {' + '.join(['1'] + selected)}")
        else:
            break  # Nenhum p-valor acima do limite → parar

    # Modelo final
    if selected:
        print("\nModelo Final Selecionado pelo Stepwise (Backward)")
        X_final = sm.add_constant(X_data[selected], prepend=True)
        final_model = sm.OLS(Y_data, X_final).fit()
        print(final_model.summary())
        return final_model
    else:
        print("Todas as variáveis foram removidas.")
        return None


In [26]:
stepwise_backward(X, Y, exit_p_value=0.15)



Procedimento Stepwise (Backward Elimination)
PS (Nível de Exclusão) = 0.15
DROP: Remove 'potencial' (P-valor: 0.4954). Modelo atual: 1 + gastos + clientes + marcas

Modelo Final Selecionado pelo Stepwise (Backward)
                            OLS Regression Results                            
Dep. Variable:               telhados   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     654.1
Date:                Wed, 05 Nov 2025   Prob (F-statistic):           1.20e-21
Time:                        22:41:29   Log-Likelihood:                -93.228
No. Observations:                  26   AIC:                             194.5
Df Residuals:                      22   BIC:                             199.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                    

## Nos MLGs - Exemplo Regressão logística


In [27]:
url='https://raw.githubusercontent.com/cibelerusso/Datasets/refs/heads/main/diabetes.csv'
df = pd.read_csv(url, sep=';')

df

,ID,Idade,Sexo,IMC,PAM,S,Y,diabetes
0,112126,32,0,27.2878,84.9978,6.0216,6.1589,0
1,897505,43,1,29.5730,82.5959,5.1257,6.9437,1
2,650952,47,1,24.3119,88.0686,3.4778,7.2397,1
3,555951,35,1,34.3610,93.8493,3.0192,7.7926,1
4,321314,54,1,26.4282,89.8297,2.2501,7.4150,1
...,...,...,...,...,...,...,...,...
145,794433,34,1,29.2444,85.3724,7.0814,7.0673,1
146,457562,37,1,24.9543,88.0610,2.7849,5.3213,0
147,450665,27,1,30.7640,87.2421,3.1033,5.5484,0
148,179491,33,1,24.6186,83.3821,3.2207,5.8325,0


In [28]:
## Forward selection

import numpy as np
import statsmodels.api as sm

def stepwise_forward_logit(X_data, Y_data, entry_p_value=0.15):
    remaining = list(X_data.columns)
    selected = []

    print("\nProcedimento Stepwise (Forward Selection - GLM Logístico)")
    print(f"PE (Nível de Inclusão) = {entry_p_value}")

    while True:
        best_p_value = np.inf
        best_feature = None

        # Testa inclusão de cada variável remanescente
        for feature in remaining:
            current_subset = selected + [feature]
            X_current = sm.add_constant(X_data[current_subset], prepend=True)

            # Evita sobreparametrização
            n, p = X_current.shape
            if p >= n:
                continue

            # Ajusta o modelo GLM Binomial (logístico)
            model = sm.GLM(Y_data, X_current, family=sm.families.Binomial()).fit()

            # P-valor da variável testada
            if feature in model.pvalues.index:
                p_value = model.pvalues[feature]
            else:
                continue

            if p_value < best_p_value:
                best_p_value = p_value
                best_feature = feature

        # Critério de inclusão (Pmin <= PE)
        if best_feature and best_p_value < entry_p_value:
            selected.append(best_feature)
            remaining.remove(best_feature)
            print(f"PASS: Adiciona '{best_feature}' (P-valor: {best_p_value:.4f}). "
                  f"Modelo atual: {' + '.join(['1'] + selected)}")
        else:
            break

    # Modelo final
    if selected:
        print("\nModelo Final Selecionado pelo Stepwise (Logístico)")
        X_final = sm.add_constant(X_data[selected], prepend=True)
        final_model = sm.GLM(Y_data, X_final, family=sm.families.Binomial()).fit()
        print(final_model.summary())
        return final_model
    else:
        print("Nenhuma variável foi selecionada.")
        return None


In [29]:
Y = df['diabetes']
X = df.drop(columns=['diabetes','Y'], axis=1)

stepwise_forward_logit(X, Y, entry_p_value=0.15)


Procedimento Stepwise (Forward Selection - GLM Logístico)
PE (Nível de Inclusão) = 0.15
PASS: Adiciona 'Idade' (P-valor: 0.0000). Modelo atual: 1 + Idade
PASS: Adiciona 'IMC' (P-valor: 0.0000). Modelo atual: 1 + Idade + IMC
PASS: Adiciona 'S' (P-valor: 0.0291). Modelo atual: 1 + Idade + IMC + S
PASS: Adiciona 'Sexo' (P-valor: 0.0269). Modelo atual: 1 + Idade + IMC + S + Sexo

Modelo Final Selecionado pelo Stepwise (Logístico)
                 Generalized Linear Model Regression Results                  
Dep. Variable:               diabetes   No. Observations:                  150
Model:                            GLM   Df Residuals:                      145
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -65.753
Date:                Wed, 05 Nov 2025   Deviance:                       131.51
Time:           

In [30]:
# Backward elimination

def stepwise_backward_logit(X_data, Y_data, exit_p_value=0.15):
    """Procedimento Stepwise (Backward Elimination) para GLM Logístico."""

    selected = list(X_data.columns)  # começa com todas as variáveis
    print("\nProcedimento Stepwise (Backward Elimination - GLM Logístico)")
    print(f"PS (Nível de Exclusão) = {exit_p_value}")

    while True:
        X_current = sm.add_constant(X_data[selected], prepend=True)
        model = sm.GLM(Y_data, X_current, family=sm.families.Binomial()).fit()

        # P-valores das variáveis (excluindo intercepto)
        pvalues = model.pvalues.iloc[1:]
        worst_feature = pvalues.idxmax()
        worst_p_value = pvalues.max()

        # Critério de saída: remove se p-valor > PS
        if worst_p_value > exit_p_value:
            selected.remove(worst_feature)
            print(f"DROP: Remove '{worst_feature}' (P-valor: {worst_p_value:.4f}). "
                  f"Modelo atual: {' + '.join(['1'] + selected)}")
        else:
            break  # nenhum p-valor acima do limiar, fim do processo

    # Modelo final
    if selected:
        print("\nModelo Final Selecionado pelo Stepwise (Backward - Logístico)")
        X_final = sm.add_constant(X_data[selected], prepend=True)
        final_model = sm.GLM(Y_data, X_final, family=sm.families.Binomial()).fit()
        print(final_model.summary())
        return final_model
    else:
        print("Todas as variáveis foram removidas.")
        return None


In [31]:
stepwise_backward_logit(X, Y, exit_p_value=0.15)


Procedimento Stepwise (Backward Elimination - GLM Logístico)
PS (Nível de Exclusão) = 0.15
DROP: Remove 'ID' (P-valor: 0.9092). Modelo atual: 1 + Idade + Sexo + IMC + PAM + S
DROP: Remove 'PAM' (P-valor: 0.3904). Modelo atual: 1 + Idade + Sexo + IMC + S

Modelo Final Selecionado pelo Stepwise (Backward - Logístico)
                 Generalized Linear Model Regression Results                  
Dep. Variable:               diabetes   No. Observations:                  150
Model:                            GLM   Df Residuals:                      145
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -65.753
Date:                Wed, 05 Nov 2025   Deviance:                       131.51
Time:                        22:41:29   Pearson chi2:                     148.
No. Iterations:                     6   Pseudo R-s